# Lecture 4 - Nested CV for CART model

## Load the CCHS data form epi7913A package

In [2]:
library(magrittr)

full_data<-epi7913A::cchs %>% dplyr::slice_sample(prop=0.1)
head(full_data)

,age,sex,CANHEARTbin,householdsize,education,maritalstatus,immigration,houseincome
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,2,1,1,3,4,1,0,1
2,1,1,1,4,1,1,0,1
3,2,2,1,2,4,1,0,1
4,7,1,1,1,4,2,0,2
5,5,2,1,2,2,3,0,1
6,5,2,0,2,4,3,0,2


## Define the outcome variable

In [3]:
# define the outcome variable
voutcome <- "CANHEARTbin"

## Iterate using 5-fold cross validation (outer loop) calling the 5-fold CV to tune the model each time (inner loop)

In [5]:
ll.mean<-mean(sapply(caret::createFolds(full_data[, voutcome], k=3), function(x)
{
  testInds <- x
  trnInds <- setdiff(1:nrow(full_data), testInds)
  
  train_data <- full_data[trnInds,] 
  test_data <- full_data[testInds,]
  
  best_model<-sdgm::cart.bestmodel.bin(train_data, voutcome, n_iter=5)

  preds<-predict(best_model, test_data)
  
  if (!is.null(preds))
  {
    test_ll<- MLmetrics::LogLoss(preds, test_data[,voutcome] ) 
  } else  {
    test_ll<-NA
    print("Logloss calculation failed")
  }
}))
print(ll.mean, na.rm=T)


 Best Parameters Found: 
Round = 6	minsplit = 5.156975	minbucket = 0.4395186	maxdepth = 6.0000	cp = -1.885992	encode = 1.0000	Value = 0.6687704 

 Best Parameters Found: 
Round = 20	minsplit = 1.449012	minbucket = 5.607885	maxdepth = 7.0000	cp = -2.946768	encode = 0.0000	Value = 0.6613827 

 Best Parameters Found: 
Round = 23	minsplit = 1.496709	minbucket = 5.215574	maxdepth = 21.0000	cp = -2.736753	encode = 1.0000	Value = 0.6590374 
[1] 0.6289018


## Now train the final model

In [6]:
  final_model<-sdgm::cart.bestmodel.bin(full_data, voutcome, n_iter=5)

[1] "Logloss on CCHS Data is: 0.622235019456334"
